## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bellevue,US,2022-04-03 19:30:25,47.6104,-122.2007,53.78,71,100,18.41,overcast clouds
1,1,Tasiilaq,GL,2022-04-03 19:34:22,65.6145,-37.6368,30.36,95,95,4.74,overcast clouds
2,2,Dingle,PH,2022-04-03 19:34:22,10.9995,122.6711,75.43,86,100,6.98,overcast clouds
3,3,Voh,NC,2022-04-03 19:34:23,-20.9667,164.7000,78.17,92,100,4.81,overcast clouds
4,4,New Norfolk,AU,2022-04-03 19:34:23,-42.7826,147.0587,56.70,89,99,1.03,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Dingle,PH,2022-04-03 19:34:22,10.9995,122.6711,75.43,86,100,6.98,overcast clouds
3,3,Voh,NC,2022-04-03 19:34:23,-20.9667,164.7000,78.17,92,100,4.81,overcast clouds
5,5,Puerto Ayora,EC,2022-04-03 19:34:23,-0.7393,-90.3518,85.96,76,45,11.61,light rain
9,9,Salalah,OM,2022-04-03 19:34:25,17.0151,54.0924,80.69,83,0,0.00,clear sky
12,12,Butaritari,KI,2022-04-03 19:34:25,3.0707,172.7902,81.46,72,98,14.50,overcast clouds
13,13,Iaciara,BR,2022-04-03 19:34:26,-14.0958,-46.6317,87.58,35,15,4.50,few clouds
24,24,Acajutla,SV,2022-04-03 19:33:33,13.5928,-89.8275,85.91,64,19,9.26,few clouds
27,27,Nazareno,MX,2022-04-03 19:34:30,25.3833,-103.4167,84.51,8,1,4.97,clear sky
29,29,Rikitea,PF,2022-04-03 19:34:30,-23.1203,-134.9692,76.71,66,56,10.27,broken clouds
30,30,Uthal,PK,2022-04-03 19:34:30,25.8072,66.6219,76.46,65,5,2.93,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                280
City                   280
Country                279
Date                   280
Lat                    280
Lng                    280
Max Temp               280
Humidity               280
Cloudiness             280
Wind Speed             280
Current Description    280
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Dingle,PH,75.43,overcast clouds,10.9995,122.6711,
3,Voh,NC,78.17,overcast clouds,-20.9667,164.7000,
5,Puerto Ayora,EC,85.96,light rain,-0.7393,-90.3518,
9,Salalah,OM,80.69,clear sky,17.0151,54.0924,
12,Butaritari,KI,81.46,overcast clouds,3.0707,172.7902,
13,Iaciara,BR,87.58,few clouds,-14.0958,-46.6317,
24,Acajutla,SV,85.91,few clouds,13.5928,-89.8275,
27,Nazareno,MX,84.51,clear sky,25.3833,-103.4167,
29,Rikitea,PF,76.71,broken clouds,-23.1203,-134.9692,
30,Uthal,PK,76.46,clear sky,25.8072,66.6219,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in {hotel_df.loc[index, 'City']} skipping.")

Hotel not found in Aitape skipping.
Hotel not found in Bauchi skipping.
Hotel not found in Mayahi skipping.
Hotel not found in Nsoko skipping.
Hotel not found in Umm Kaddadah skipping.
Hotel not found in Bondo skipping.
Hotel not found in Madimba skipping.
Hotel not found in Tanout skipping.
Hotel not found in Wattegama skipping.
Hotel not found in Avera skipping.
Hotel not found in Gamba skipping.
Hotel not found in Manubul skipping.
Hotel not found in Birao skipping.
Hotel not found in Nioro skipping.
Hotel not found in Sembe skipping.
Hotel not found in Malinyi skipping.
Hotel not found in Sebastian skipping.
Hotel not found in Masuguru skipping.
Hotel not found in Merauke skipping.
Hotel not found in Sembakung skipping.
Hotel not found in Micheweni skipping.
Hotel not found in Aripuana skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [17]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))